<h1> Deploying and predicting with model </h1>

<h2>Learning Objectives</h2>
<ol>
    <li>Create the model using ai-platform CLI commands</li>
<li>Deploy the ML model to production</li>
    <li>Perform predictions with the model</li>
</ol>

__TODO__: Complete the lab notebook #TODO sections. You can refer to the [solutions/](../solutions/6_deploy.ipynb) notebook for reference. 


In [ ]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.13'  

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/babyweight/trained_model; then
  gsutil mb -l ${REGION} gs://${BUCKET}
  # copy canonical model if you didn't do previous notebook
  gsutil -m cp -R gs://cloud-training-demos/babyweight/trained_model gs://${BUCKET}/babyweight
fi

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/

In [ ]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"

# Optional: Delete the version of the model if it already exists:
#gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ai-platform models delete ${MODEL_NAME}

# TODO: Create the model 
gcloud ai-platform models create 

# TODO: Create the model version 
gcloud ai-platform versions create

<h2> Use model to predict (online prediction) </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

In [ ]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = 'babyweight'
MODEL_VERSION = 'ml_on_gcp'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'key': 'b1',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 39
    },
    {
      'key': 'g1',
      'is_male': 'False',
      'mother_age': 29.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 38
    },
    {
      'key': 'b2',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Triplets(3)',
      'gestation_weeks': 39
    },
    {
      'key': 'u1',
      'is_male': 'Unknown',
      'mother_age': 29.0,
      'plurality': 'Multiple(2+)',
      'gestation_weeks': 38
    },
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

The predictions for the four instances were: 7.66, 7.22, 6.32 and 6.19 pounds respectively when I ran it (your results might be different).

<h2> Use model to predict (batch prediction) </h2>
<p>
Batch prediction is commonly used when you thousands to millions of predictions.
Create a file with one instance per line and submit using gcloud.

In [ ]:
%%writefile inputs.json
{"key": "b1", "is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"key": "g1", "is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

In [1]:
%%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 

gcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=babyweight --version=ml_on_gcp

CommandException: "cp" command does not support provider-only URLs.
CommandException: "rm" command does not support provider-only URLs.
ERROR: (gcloud.ai-platform.jobs.submit.prediction) argument --region: expected one argument
Usage: gcloud ai-platform jobs submit prediction JOB --data-format=DATA_FORMAT --input-paths=INPUT_PATH,[INPUT_PATH,...] --output-path=OUTPUT_PATH --region=REGION (--model=MODEL | --model-dir=MODEL_DIR) [optional flags]
  optional flags may be  --batch-size | --help | --labels | --max-worker-count |
                         --model | --model-dir | --runtime-version |
                         --signature-name | --version

For detailed information on this command and its flags, run:
  gcloud ai-platform jobs submit prediction --help


CalledProcessError: Command 'b'INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json\nOUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs\ngsutil cp inputs.json $INPUT\ngsutil -m rm -rf $OUTPUT \n\ngcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \\\n  --data-format=TEXT --region ${REGION} \\\n  --input-paths=$INPUT \\\n  --output-path=$OUTPUT \\\n  --model=babyweight --version=ml_on_gcp\n'' returned non-zero exit status 2

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License